In [5]:
import os
import random
import json
import time
from io import BytesIO
import cv2
import base64
import logging
import uuid
import traitlets
import threading
import numpy as np
from gym_donkeycar.core.sim_client import SDClient


###########################################

class SimpleClient(SDClient, traitlets.HasTraits):
    image = traitlets.Any()
    imageB = traitlets.Any()
    throtle = traitlets.Float()
    stearing = traitlets.Float()
    message = traitlets.Unicode()
    driving = traitlets.Bool(default_value=False)
    
    def pipeline(self, src):
        zero = np.zeros([src.shape[0], src.shape[1]])
        blue = np.int16(np.matrix(src[:,:,0]))
        green = np.int16(np.matrix(src[:,:,1]))
        red = np.int16(np.matrix(src[:,:,2]))
        red_only = red-blue
        green_only = green-blue
        red_only[red_only<0] = 0
        red_only[red_only>255] = 255
        red_only = np.uint8(red_only)
        green_only[green_only<0] = 0
        green_only[green_only>255] = 255
        green_only = np.uint8(green_only)
        image =  green_only + red_only
        image[150:255,:] = 0
        contours, hierarchy = cv2.findContours(image ,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        c = max(contours, key=cv2.contourArea)

        M = cv2.moments(c)

        cx = int(M['m10']/M['m00'])

        cy = int(M['m01']/M['m00'])

        cv2.line(image,(cx,0),(cx,255),255,1)

        cv2.line(image,(0,cy),(255,cy),255,1)
        cv2.drawContours(image,[c],-1,255,3)
        center = 128
        error = center - cx
        out = error * 2.5
        if out > center:
            out = center
        elif out < -center:
            out = -center
        #self.stearing = (1 - (cx / 255.0))
        self.stearing = out / center
        return image
    
    def __init__(self, address, racer, vehicle, poll_socket_sleep_time=0.01, *args, **kwargs):
        host, port = address
        traitlets.HasTraits.__init__(self, *args, **kwargs)
        SDClient.__init__(self, host, port, poll_socket_sleep_time=poll_socket_sleep_time)
        self.last_image = None
        self.car_loaded = False
        self.racer = racer
        self.vehicle = vehicle
        self._driving = False

    def on_msg_recv(self, json_packet):
        
        if json_packet['msg_type'] == "need_car_config":
            self.message = 'Sending config'
            self.send_config(self.racer, self.vehicle)

        if json_packet['msg_type'] == "car_loaded":
            self.message = 'Car is loaded'
            self.car_loaded = True
            
        
        if json_packet['msg_type'] == "telemetry":
            imgString = json_packet["image"]
            
            self.imageB = cv2.imdecode(np.frombuffer(base64.b64decode(imgString), dtype=np.uint8), cv2.IMREAD_COLOR)
            self.image = self.pipeline(self.imageB)

            #don't have to, but to clean up the print, delete the image string.
            del json_packet["image"]

            if "imageb" in json_packet:
                print('got image b')
                self.imageB = cv2.imdecode(np.frombuffer(base64.b64decode(imgString), dtype=np.uint8), cv2.IMREAD_COLOR)
                #don't have to, but to clean up the print, delete the image string.
                del json_packet["imageb"]

            if "lidar" in json_packet:
                lidar = json_packet["lidar"]
                if lidar is not None:
                    print("lidar:", len(lidar), "pts")

                #don't have to, but to clean up the print, delete the lidar string.
                del json_packet["lidar"]
                
        else:
            self.message = str(json_packet)


    def send_config(self, racer, vehicle):
        '''
        send three config messages to setup car, racer, and camera
        '''
        racer_name = racer
        car_name = vehicle
        bio = "I race robots."
        country = "France"
        guid = str(uuid.uuid4())

        # Racer info
        msg = {'msg_type': 'racer_info',
            'racer_name': racer_name,
            'car_name' : car_name,
            'bio' : bio,
            'country' : country,
            'guid' : guid }
        self.send_now(json.dumps(msg))
        time.sleep(0.2)
        
        # Car config
        # body_style = "donkey" | "bare" | "car01" choice of string
        # body_rgb  = (128, 128, 128) tuple of ints
        # car_name = "string less than 64 char"

        msg = '{ "msg_type" : "car_config", "body_style" : "donkey", "body_r" : "255", "body_g" : "0", "body_b" : "255", "car_name" : "%s", "font_size" : "100" }' % (car_name)
        self.send_now(msg)

        #this sleep gives the car time to spawn. Once it's spawned, it's ready for the camera config.
        time.sleep(0.2)
 
        msg = '''{
    "msg_type" : "cam_config",
    "fov" : "150", 
    "fish_eye_x" : "0.0",
    "fish_eye_y" : "0.0",
    "img_w" : "255",
    "img_h" : "255",
    "img_d" : "3",
    "img_enc" : "JPG",
    "offset_x" : "0.0",
    "offset_y" : "1.0",
    "offset_z" : "2.0",
    "rot_x" : "120.0"
    }'''
        self.send_now(msg)


    def start_driving(self):
        self.driving = True
        
    def stop_driving(self):
        self.driving = False
        
        
        
    def send_controls(self, steering, throttle):
        msg = { "msg_type" : "control",
                "steering" : steering.__str__(),
                "throttle" : throttle.__str__(),
                "brake" : "0.0" }
        self.send(json.dumps(msg))

        #this sleep lets the SDClient thread poll our message and send it out.
        time.sleep(self.poll_socket_sleep_sec)
        
    def request_scene(self):
        msg = '{ "msg_type" : "load_scene", "scene_name" : "mountain_track" }'
        self.send_now(msg)
        
    def exit_scene(self):
        if self._driving:
            self.stop_driving()
        msg = '{ "msg_type" : "exit_scene" }'
        self.send_now(msg)
        
    def update(self):
        #st = ((self.stearing * 2) - 1)
        st = self.stearing
        th = ((1 - abs(st)) * 0.2) + 0.1
        self.send_controls(st, th)
        
    def _send_updates(self):
        while True:
            if not self._driving:
                break
            if not self.car_loaded:
                time.sleep(0.3)
                continue
            self.update()
            if self.aborted:
                print("Client socket problem, stopping driving.")
                self.driving = False
            
    @traitlets.observe('driving')
    def _on_driving(self, change):
        if change['new'] and not change['old']:
            # transition from not driving -> driving
            self._driving = True
            self.thread = threading.Thread(target=self._send_updates)
            self.thread.start()
        elif change['old'] and not change['new']:
            # transition from driving -> not driving
            self._driving = False
            self.thread.join()
    

In [6]:
from ipywidgets import Layout, Image, HBox, VBox, Output, Button
from IPython.display import display

image_widget = Image(format='jpeg', layout=Layout(width='500px', height='500px'))

image_widgetb = Image(format='jpeg', layout=Layout(width='500px', height='500px'))

output = Output()

load_scene = Button(description="Load scene")

drive = Button(description="Drive !")

stop = Button(description="Stop !")

exit = Button(description="Exit !")

row1 = HBox([image_widget, image_widgetb])

row2 = HBox([load_scene, drive, stop, exit])

row3 = HBox([output])

box = VBox([row1, row2, row3])

In [7]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [8]:
display(box)

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(cv2.flip(image,-1))

def update_imageb(change):
    image = change['new']
    image_widgetb.value = bgr8_to_jpeg(cv2.flip(image,-1))
    
def update_message(change):
    message = change['new']
    output.clear_output()
    with output:
        print(message)
        



logging.basicConfig(level=logging.INFO)

# test params
host = "127.0.0.1" # "trainmydonkey.com" for virtual racing server
port = 9091
time_to_drive = 120.0


c = SimpleClient((host, port), "Pedro3", "Twingo")
c.observe(update_image, names='image')
c.observe(update_imageb, names='imageB')
c.observe(update_message, names='message')

load_scene.on_click(lambda r: c.request_scene())

drive.on_click(lambda r: c.start_driving())

stop.on_click(lambda r: c.stop_driving())

exit.on_click(lambda r: c.exit_scene())


INFO:gym_donkeycar.core.client:connecting to 127.0.0.1:9091 


In [5]:
print('footer')

footer
